In [1]:
import glob
import numpy
import pickle
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
# from keras.layers import BatchNormalization 
from tensorflow.keras.layers import BatchNormalization


from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


In [2]:
def prepare_notes():
    # prepare notes from midi file
    notes = []
    for file in glob.glob("./datasets/MIDI/*.mid"):
        midi = converter.parse(file)
#         print("Parsing file: %s", file)
        notes_to_parse = None
        
        #file has instrument parts
        try:
            p_ins = instrument.partitionByInstrument(midi)
            notes_to_parse = p_ins.parts[0].recurse()
        except:
            notes_to_parse = midi.flat.notes
            
        #populate notes
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    
    return notes
                
                
        

In [3]:
def train_network():
    notes = prepare_notes()
    n_vocab = len(set(notes))
    network_ip, network_op = prepare_sequences(notes, n_vocab)
    model = create_network(network_ip, n_vocab)
    train(model, network_ip, network_op)
    

In [4]:
def prepare_sequences(notes, n_vocab):
    sequence_length = 100
    
    #pitch name
    pitchnames = sorted(set(item for item in notes))
    
    #map pitch to int
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    network_ip = []
    network_op = []
    
    # create ip and op sequences
    for i in range(0, len(notes)-sequence_length,1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_ip.append([note_to_int[char] for char in sequence_in])
        network_op.append(note_to_int[sequence_out])
        
    n_patterns = len(network_ip)
    
    network_ip = numpy.reshape(network_ip,(n_patterns,sequence_length,1))
    network_ip = network_ip/float(n_vocab)
    network_op = np_utils.to_categorical(network_op)
    
    return(network_ip, network_op)

In [8]:
def create_network(network_ip, n_vocab):
    model = Sequential()
    model.add(LSTM(512,
                  input_shape=(network_ip.shape[1], network_ip.shape[2]),
                  recurrent_dropout=0.1,
                  return_sequences=True
                  ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3))
    model.add(LSTM(512))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    return model
    

In [9]:
def train(model, network_ip, network_op):
    filepath ="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    model.fit(network_ip, network_op, epochs=10, batch_size=128, callbacks=callbacks_list)

In [ ]:
if __name__ == '__main__':
    train_network()

Epoch 1/10
346/346 [==============================] - 3961s 11s/step - loss: 5.1797
Epoch 2/10
 62/346 [====>.........................] - ETA: 3:13:32 - loss: 4.5823